<a href="https://colab.research.google.com/github/MStamirski/365DataScience-students/blob/main/FeaturesEngineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation, data cleaning

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
path = "drive/MyDrive/Colab_Notebooks/365Challenge/data/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import datetime

In [ ]:
filenames = ['365_course_info', '365_course_ratings', '365_exam_info', '365_quiz_info', '365_student_engagement', '365_student_exams', '365_student_hub_questions', '365_student_info', '365_student_learning', '365_student_purchases', '365_student_quizzes']

In [ ]:
def read_data(filename):
  df = pd.read_csv(path+filename+".csv")
  for col in df.columns:
    if 'date' in col:
      df[col] = pd.to_datetime(df[col], format='%Y-%m-%d')
  return df

In [ ]:
src_course_info = read_data(filenames[0])
src_course_rating = read_data(filenames[1])
src_exam_info = read_data(filenames[2])
src_quiz_info = read_data(filenames[3])
src_student_engagement = read_data(filenames[4])
src_student_exams = read_data(filenames[5])

In [ ]:
src_student_exams.shape

(34030, 6)

In [ ]:
src_student_exams['exam_completion_time'].max()

198.12

In [ ]:
quant = np.quantile(src_student_exams['exam_completion_time'],0.99)
quant

75.0

In [ ]:
# About 1% of data are outliers, let's assign it to a value of 99 quantile
src_student_exams['exam_completion_time'] = src_student_exams['exam_completion_time'].apply(lambda x: min(x, quant))

In [ ]:
src_student_exams['exam_completion_time'].describe()

count    34030.000000
mean         8.316957
std         12.360311
min          0.070000
25%          1.030000
50%          4.225000
75%         10.000000
max         75.000000
Name: exam_completion_time, dtype: float64

In [ ]:
src_student_hub_questions = read_data(filenames[6])
src_student_info = read_data(filenames[7])
src_student_learning = read_data(filenames[8])

In [ ]:
src_student_learning.shape

(64535, 4)

In [ ]:
src_student_learning['minutes_watched'].describe()

count    64535.000000
mean        28.545392
std         40.580553
min          0.000000
25%          3.800000
50%         15.900000
75%         38.100000
max       1710.600000
Name: minutes_watched, dtype: float64

In [ ]:
quant = np.quantile(src_student_learning['minutes_watched'],0.99)
quant

177.4

In [ ]:
# About 1% of data are outliers, let's assign it to a value of 99 quantile
src_student_learning['minutes_watched'] = src_student_learning['minutes_watched'].apply(lambda x: min(x, quant))

In [ ]:
src_student_learning['minutes_watched'].describe()

count    64535.000000
mean        27.719826
std         34.008315
min          0.000000
25%          3.800000
50%         15.900000
75%         38.100000
max        177.400000
Name: minutes_watched, dtype: float64

In [ ]:
src_student_purchases = read_data(filenames[9])
src_student_quizzes = read_data(filenames[10])

In [ ]:
src_student_quizzes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147029 entries, 0 to 147028
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   student_id   147029 non-null  int64  
 1   quiz_id      147029 non-null  int64  
 2   question_id  147029 non-null  int64  
 3   answer_id    146800 non-null  float64
dtypes: float64(1), int64(3)
memory usage: 4.5 MB


In [ ]:
src_student_quizzes['answer_id'].describe()

count    146800.000000
mean       2613.594149
std        1298.666048
min          85.000000
25%        1420.000000
50%        3087.000000
75%        3267.000000
max        4755.000000
Name: answer_id, dtype: float64

In [ ]:
# answer "0" will be treated as wrong answer
src_student_quizzes['answer_id'] = src_student_quizzes['answer_id'].fillna(0)

In [ ]:
src_student_quizzes['answer_id'] = src_student_quizzes['answer_id'].astype(int)

In [ ]:
src_student_quizzes['answer_id'].describe()

count    147029.000000
mean       2609.523434
std        1301.740903
min           0.000000
25%        1420.000000
50%        3087.000000
75%        3267.000000
max        4755.000000
Name: answer_id, dtype: float64

In [ ]:
src_student_quizzes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147029 entries, 0 to 147028
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype
---  ------       --------------   -----
 0   student_id   147029 non-null  int64
 1   quiz_id      147029 non-null  int64
 2   question_id  147029 non-null  int64
 3   answer_id    147029 non-null  int64
dtypes: int64(4)
memory usage: 4.5 MB


In [ ]:
# function for filtering data: up to date of first payment or all of them (up to future date)
def get_payment_date(student_id):
  dates = src_student_purchases[src_student_purchases['student_id']==student_id]['date_purchased']
  if len(dates) == 0:
    return datetime.date(2022, 12, 31)
  else:
    return dates.min().date()

# creating dataset: courses

In [ ]:
# one-hot-encoding
def make_one_hot(categories, assignments):
  dict_cat = {}
  categories = sorted(categories)
  for category in categories:
    if category in assignments.keys():
      dict_cat[category] = assignments[category]
    else:
      dict_cat[category] = 0
  return dict_cat

In [ ]:
# list of passed unique courses id's for student before payment
def get_courses_id(student_id, end_date):
  student_courses = src_student_learning[src_student_learning['student_id']==student_id]
  courses_to_date = student_courses[student_courses['date_watched'].dt.date<end_date]['course_id']
  return courses_to_date.unique().tolist()

In [ ]:
# one-hot-encoding to dataset columns of passed courses id's for student before payment
def get_one_hot_courses_id(student_id):
  end_date = get_payment_date(student_id)
  selected_courses = get_courses_id(student_id, end_date)
  dictionary_courses_id = {course: 1 for course in selected_courses}
  courses_list = src_course_info['course_id'].sort_values()
  one_hot_courses_id = make_one_hot(courses_list, dictionary_courses_id)
  one_hot_courses_id_cols = {"Course_Nr_"+str(course): one_hot_courses_id[course] for course in one_hot_courses_id.keys()}
  return one_hot_courses_id_cols

In [ ]:
def get_total_courses_nr(student_id):
  end_date = get_payment_date(student_id)
  selected_courses = get_courses_id(student_id, end_date)
  return {"Courses_Nr": len(selected_courses)}

In [ ]:
# dictionary of time in passed courses for student before payment
def get_courses_time(student_id, end_date):
  student_courses = src_student_learning[src_student_learning['student_id']==student_id]
  courses_to_date = student_courses[student_courses['date_watched'].dt.date<end_date]
  times_to_date = courses_to_date.groupby("course_id").sum()
  times_in_courses = {}
  for course, minutes in zip(times_to_date.index, times_to_date['minutes_watched']):
    times_in_courses[course] = minutes
  return times_in_courses

In [ ]:
# one-hot-encoding to dataset columns of time in passed courses for student before payment
def get_one_hot_courses_time(student_id):
  end_date = get_payment_date(student_id)
  dictionary_times = get_courses_time(student_id, end_date)
  courses_list = src_course_info['course_id'].sort_values()
  one_hot_courses_time = make_one_hot(courses_list, dictionary_times)
  one_hot_courses_time_cols = {"Course_Time_"+str(course): one_hot_courses_time[course] for course in one_hot_courses_time.keys()}
  return one_hot_courses_time_cols

# end_date = datetime.date(2022,12,31)
# get_one_hot_courses_time(266862)

In [ ]:
def get_total_courses_time(student_id):
  end_date = get_payment_date(student_id)
  dictionary_times = get_courses_time(student_id, end_date)
  return {"Courses_Time": sum(dictionary_times.values())}

In [ ]:
# general rating of student courses before his payment
def get_courses_general_rating(student_id, end_date):
  student_courses = src_student_learning[src_student_learning['student_id']==student_id]
  courses_to_date = student_courses[student_courses['date_watched'].dt.date<end_date]["course_id"]
  ratings = src_course_rating[src_course_rating['date_rated'].dt.date<end_date].groupby("course_id").mean()
  ratings_of_courses = {}
  for course in courses_to_date:
    if course in ratings.index:
      ratings_of_courses[course] = ratings.loc[course]["course_rating"]
    else:
      ratings_of_courses[course] = 0
  return ratings_of_courses

In [ ]:
# one-hot-encoding to dataset columns of average general ratings of courses passed by student before his payment
def get_one_hot_courses_general_rating(student_id):
  end_date = get_payment_date(student_id)
  dictionary_rating = get_courses_general_rating(student_id, end_date)
  courses_list = src_course_info['course_id'].sort_values()
  one_hot_courses_general_ratings = make_one_hot(courses_list, dictionary_rating)
  one_hot_courses_general_ratings_cols = {"Course_GeneralRating_"+str(course): one_hot_courses_general_ratings[course] for course in one_hot_courses_general_ratings.keys()}
  return one_hot_courses_general_ratings_cols

# end_date = datetime.date(2022,12,31)
# get_one_hot_courses_general_rating(272965)

In [ ]:
def get_total_average_general_rating(student_id):
  end_date = get_payment_date(student_id)
  dictionary_rating = get_courses_general_rating(student_id, end_date)
  if len(dictionary_rating.values())==0:
    rat = 0
  else:
    rat = sum(dictionary_rating.values())/len(dictionary_rating.values())
  return {"Courses_GeneralRating": rat}

In [ ]:
# individual rating of student courses before his payment
def get_courses_individual_rating(student_id, end_date):
  student_courses = src_student_learning[src_student_learning['student_id']==student_id]
  courses_to_date = student_courses[student_courses['date_watched'].dt.date<end_date]["course_id"]
  individual_ratings = src_course_rating[src_course_rating['student_id']==student_id]
  mean_individual_ratings = individual_ratings[individual_ratings['date_rated'].dt.date<end_date].groupby("course_id").mean()
  dict_general_ratings = get_courses_general_rating(student_id, end_date)
  ratings_of_courses = {}
  for course in courses_to_date:
    if course in mean_individual_ratings.index:
      ratings_of_courses[course] = mean_individual_ratings.loc[course]["course_rating"]
    else:
      ratings_of_courses[course] = dict_general_ratings[course]
  return ratings_of_courses

In [ ]:
# one-hot-encoding to dataset columns of average individual ratings of courses passed by student before his payment
def get_one_hot_courses_individual_rating(student_id):
  end_date = get_payment_date(student_id)
  dictionary_rating = get_courses_individual_rating(student_id, end_date)
  courses_list = src_course_info['course_id'].sort_values()
  one_hot_courses_individual_ratings = make_one_hot(courses_list, dictionary_rating)
  one_hot_courses_individual_ratings_cols = {"Course_IndividRating_"+str(course): one_hot_courses_individual_ratings[course] for course in one_hot_courses_individual_ratings.keys()}
  return one_hot_courses_individual_ratings_cols

# end_date = datetime.date(2022,12,31)
# get_one_hot_courses_individual_rating(272965)

In [ ]:
def get_total_average_individual_rating(student_id):
  end_date = get_payment_date(student_id)
  dictionary_rating = get_courses_individual_rating(student_id, end_date)
  if len(dictionary_rating.values())==0:
    rat = 0
  else:
    rat = sum(dictionary_rating.values())/len(dictionary_rating.values())
  return {"Courses_IndividRating": rat}

In [ ]:
def make_courses_dataset():
  records = []
  students = src_student_info['student_id']
  for student in students:
    courses_numbers = get_one_hot_courses_id(student)
    courses_total_number = get_total_courses_nr(student)
    courses_times = get_one_hot_courses_time(student)
    courses_total_time = get_total_courses_time(student)
    courses_general_ratings = get_one_hot_courses_general_rating(student)
    courses_total_general_rating = get_total_average_general_rating(student)
    courses_individual_ratings = get_one_hot_courses_individual_rating(student)
    courses_total_individual_rating = get_total_average_individual_rating(student)

    columns = {"student_id": student}
    columns.update(courses_numbers)
    columns.update(courses_total_number)
    columns.update(courses_times)
    columns.update(courses_total_time)
    columns.update(courses_general_ratings)
    columns.update(courses_total_general_rating)
    columns.update(courses_individual_ratings)
    columns.update(courses_total_individual_rating)

    records.append(columns)

  df = pd.DataFrame(records)
  return df

In [ ]:
Courses = make_courses_dataset()
Courses

,student_id,Course_Nr_2,Course_Nr_3,Course_Nr_4,Course_Nr_5,Course_Nr_7,Course_Nr_11,Course_Nr_12,Course_Nr_13,Course_Nr_14,...,Course_IndividRating_49,Course_IndividRating_50,Course_IndividRating_51,Course_IndividRating_52,Course_IndividRating_53,Course_IndividRating_54,Course_IndividRating_55,Course_IndividRating_56,Course_IndividRating_57,Courses_IndividRating
0,258798,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,4.776860
1,258799,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.000000
2,258800,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.000000
3,258801,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.000000
4,258802,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35225,295511,0,0,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,4.858392
35226,295512,0,0,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,4.858392
35227,295513,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,4.813953
35228,295514,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.000000


In [ ]:
Courses.to_csv(path+"courses.csv", sep="\t")

In [ ]:
Courses.describe()

,student_id,Course_Nr_2,Course_Nr_3,Course_Nr_4,Course_Nr_5,Course_Nr_7,Course_Nr_11,Course_Nr_12,Course_Nr_13,Course_Nr_14,...,Course_IndividRating_49,Course_IndividRating_50,Course_IndividRating_51,Course_IndividRating_52,Course_IndividRating_53,Course_IndividRating_54,Course_IndividRating_55,Course_IndividRating_56,Course_IndividRating_57,Courses_IndividRating
count,35230.000000,35230.000000,35230.000000,35230.000000,35230.000000,35230.000000,35230.000000,35230.000000,35230.000000,35230.000000,...,35230.000000,35230.000000,35230.000000,35230.000000,35230.000000,35230.000000,35230.000000,35230.0,35230.000000,35230.000000
mean,277003.640874,0.011013,0.009991,0.004485,0.003463,0.210843,0.005932,0.003974,0.014845,0.030883,...,0.005081,0.007067,0.005176,0.004104,0.013469,0.003125,0.002426,0.0,0.000596,2.247895
std,10556.258459,0.104366,0.099458,0.066819,0.058746,0.407913,0.076795,0.062914,0.120935,0.173003,...,0.156769,0.182125,0.155479,0.140581,0.252636,0.122291,0.107309,0.0,0.024408,2.397615
min,258798.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
25%,267815.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
50%,277001.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
75%,286075.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,4.817626
max,295515.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,5.000000,5.000000,5.000000,4.857143,5.000000,5.000000,5.000000,0.0,1.000000,5.000000


In [ ]:
for col in Courses.columns: print(Courses[col].describe())

count     35230.000000
mean     277003.640874
std       10556.258459
min      258798.000000
25%      267815.250000
50%      277001.500000
75%      286075.750000
max      295515.000000
Name: student_id, dtype: float64
count    35230.000000
mean         0.011013
std          0.104366
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: Course_Nr_2, dtype: float64
count    35230.000000
mean         0.009991
std          0.099458
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: Course_Nr_3, dtype: float64
count    35230.000000
mean         0.004485
std          0.066819
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: Course_Nr_4, dtype: float64
count    35230.000000
mean         0.003463
std          0.058746
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000

In [ ]:
for col in Courses.columns: print(Courses[col].value_counts())

258798    1
283091    1
283085    1
283086    1
283087    1
         ..
270900    1
270899    1
270898    1
270897    1
295515    1
Name: student_id, Length: 35230, dtype: int64
0    34842
1      388
Name: Course_Nr_2, dtype: int64
0    34878
1      352
Name: Course_Nr_3, dtype: int64
0    35072
1      158
Name: Course_Nr_4, dtype: int64
0    35108
1      122
Name: Course_Nr_5, dtype: int64
0    27802
1     7428
Name: Course_Nr_7, dtype: int64
0    35021
1      209
Name: Course_Nr_11, dtype: int64
0    35090
1      140
Name: Course_Nr_12, dtype: int64
0    34707
1      523
Name: Course_Nr_13, dtype: int64
0    34142
1     1088
Name: Course_Nr_14, dtype: int64
0    34868
1      362
Name: Course_Nr_15, dtype: int64
0    34554
1      676
Name: Course_Nr_16, dtype: int64
0    35128
1      102
Name: Course_Nr_19, dtype: int64
0    34857
1      373
Name: Course_Nr_20, dtype: int64
0    34055
1     1175
Name: Course_Nr_21, dtype: int64
0    34996
1      234
Name: Course_Nr_22, dtype: int64
0 

# creatinng dataset: quizzes

In [ ]:
# there are no dates for specific quizes available
def get_quiz_result(student_id, quiz_id):
  quiz = src_quiz_info[src_quiz_info['quiz_id']==quiz_id]
  quiz_questions = quiz['question_id'].sort_values().unique()
  correct_answers = []
  for question in quiz_questions:
    answers = quiz[quiz['question_id']==question]
    answer = answers[answers['answer_correct']=='y']['answer_id']
    correct_answers.append(answer.values[0])

  student_quiz = src_student_quizzes[src_student_quizzes['student_id']==student_id]
  student_questions = student_quiz[student_quiz['quiz_id']==quiz_id]
  given_answers = []
  for question in quiz_questions:
    answer = student_questions[student_questions['question_id']==question]['answer_id']
    if len(answer.values)>0:
      given_answers.append(answer.values[0])
    else:
      given_answers.append(0)
  
  total_answers = 0
  good_answers = 0
  for correct, given in zip(correct_answers, given_answers):
    if correct==given:
      good_answers += 1
    total_answers += 1

  return good_answers / total_answers

In [ ]:
def make_quizes_dataset():
  records = []
  students = src_student_info['student_id']

  for student in students:
    quizes = src_student_quizzes[src_student_quizzes['student_id']==student]['quiz_id'].unique()
    result_sum = 0
    for quiz in quizes:
        result_sum += get_quiz_result(student, quiz)
    
    if len(quizes)==0:
      result_avg = 0
    else:
      result_avg = result_sum / len(quizes)

    columns = {"student_id": student}
    columns.update({"Quizes_Result": result_avg})
    records.append(columns)

  df = pd.DataFrame(records)
  return df

In [ ]:
Quizes = make_quizes_dataset()
Quizes

,student_id,Quizes_Result
0,258798,0.000000
1,258799,0.000000
2,258800,0.551918
3,258801,0.000000
4,258802,0.000000
...,...,...
35225,295511,0.000000
35226,295512,0.533333
35227,295513,0.457143
35228,295514,0.000000


In [ ]:
for col in Quizes.columns: print(Quizes[col].describe())

count     35230.000000
mean     277003.640874
std       10556.258459
min      258798.000000
25%      267815.250000
50%      277001.500000
75%      286075.750000
max      295515.000000
Name: student_id, dtype: float64
count    35230.000000
mean         0.137729
std          0.282596
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: Quizes_Result, dtype: float64


In [ ]:
for col in Quizes.columns: print(Quizes[col].value_counts())

258798    1
283091    1
283085    1
283086    1
283087    1
         ..
270900    1
270899    1
270898    1
270897    1
295515    1
Name: student_id, Length: 35230, dtype: int64
0.000000    27997
0.500000      786
0.785714      667
0.750000      653
0.833333      308
            ...  
0.715107        1
0.803030        1
0.669259        1
0.723767        1
0.626190        1
Name: Quizes_Result, Length: 1772, dtype: int64


In [ ]:
Quizes.to_csv(path+"quizes.csv", sep="\t")

# creating dataset: engagement

In [ ]:
def get_engagement(student_id, end_date):
  engagements = src_student_engagement[src_student_engagement['student_id']==student_id]
  engagements_to_date = engagements[engagements['date_engaged'].dt.date<end_date]
  quizes_sum = engagements_to_date['engagement_quizzes'].sum()
  exams_sum = engagements_to_date['engagement_exams'].sum()
  lessons_sum = engagements_to_date['engagement_lessons'].sum()
  return quizes_sum, exams_sum, lessons_sum

In [ ]:
def make_engagements_dataset():
  records = []
  students = src_student_info['student_id']

  for student in students:
    end_date = get_payment_date(student)
    quiz, exam, lesson = get_engagement(student, end_date)

    columns = {"student_id": student}
    columns.update({"Engagement_quiz": quiz})
    columns.update({"Engagement_exam": exam})
    columns.update({"Engagement_lesson": lesson})
    records.append(columns)

  df = pd.DataFrame(records)
  return df

In [ ]:
Engagements = make_engagements_dataset()
Engagements

,student_id,Engagement_quiz,Engagement_exam,Engagement_lesson
0,258798,0,0,1
1,258799,0,0,0
2,258800,0,0,0
3,258801,0,0,0
4,258802,0,0,0
...,...,...,...,...
35225,295511,1,0,1
35226,295512,1,0,1
35227,295513,0,0,1
35228,295514,0,0,0


In [ ]:
Engagements.to_csv(path+"engagements.csv", sep="\t")

In [ ]:
for col in Engagements.columns: print(Engagements[col].describe())

count     35230.000000
mean     277003.640874
std       10556.258459
min      258798.000000
25%      267815.250000
50%      277001.500000
75%      286075.750000
max      295515.000000
Name: student_id, dtype: float64
count    35230.000000
mean         0.279762
std          0.807841
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         29.000000
Name: Engagement_quiz, dtype: float64
count    35230.000000
mean         0.100852
std          0.512266
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         28.000000
Name: Engagement_exam, dtype: float64
count    35230.000000
mean         0.820579
std          1.619508
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max         88.000000
Name: Engagement_lesson, dtype: float64


In [ ]:
for col in Engagements.columns: print(Engagements[col].value_counts())

258798    1
283091    1
283085    1
283086    1
283087    1
         ..
270900    1
270899    1
270898    1
270897    1
295515    1
Name: student_id, Length: 35230, dtype: int64
0     28924
1      4275
2      1289
3       441
4       158
5        72
6        19
7        12
10        7
8         6
11        6
9         6
15        4
16        3
18        2
13        2
14        1
20        1
29        1
19        1
Name: Engagement_quiz, dtype: int64
0     32714
1      1969
2       351
3       108
4        45
5        13
6         9
7         5
11        3
14        3
9         2
10        2
12        2
28        1
13        1
27        1
19        1
Name: Engagement_exam, dtype: int64
0     18302
1     11700
2      2825
3      1082
4       544
5       292
6       147
7       100
8        56
9        46
11       22
10       22
12       18
14       13
13       10
15        7
18        6
25        4
19        4
16        4
17        3
23        3
20        3
27        2
22        2
47    

# creating dataset: exams

In [ ]:
def get_exam_dictionaries(student_id, end_date):
  student_attempts = src_student_exams[src_student_exams['student_id']==student_id]
  attempts_to_date = student_attempts[student_attempts['date_exam_completed'].dt.date<end_date]
  cat1 = cat2 = cat4 = 0
  time1 = time2 = time4 = 0
  dura1 = dura2 = dura4 = 0
  res1 = res2 = res4 = 0
  mean_duration = src_exam_info['exam_duration'].mean()

  for exam, time, result in zip(attempts_to_date['exam_id'].tolist(), attempts_to_date['exam_completion_time'].tolist(), attempts_to_date['exam_result'].tolist()):
    if len(src_exam_info[src_exam_info['exam_id']==exam])>0:
      exam_category = src_exam_info[src_exam_info['exam_id']==exam]['exam_category'].values[0]
      exam_duration = src_exam_info[src_exam_info['exam_id']==exam]['exam_duration'].values[0]
    else:
      # in some cases there was no corresponding records in "exam_info" dataset
      exam_category = 2
      exam_duration = mean_duration

    if exam_category==1:
      cat1 += 1
      time1 += time
      dura1 += exam_duration
      res1 += result
    elif exam_category==2:
      cat2 += 1
      time2 += time
      dura2 += exam_duration
      res2 += result
    else:
      cat4 += 1
      time4 += time
      dura4 += exam_duration
      res4 += result

  if cat1==0: 
    avgtime1 = 0 
    avgperc1 = 0
    avgres1 = 0
  else: 
    avgtime1 = time1 / cat1
    avgperc1 = time1 / dura1
    avgres1 = res1 / cat1
  
  if cat2==0: 
    avgtime2 = 0 
    avgperc2 = 0
    avgres2 = 0
  else: 
    avgtime2 = time2 / cat2
    avgperc2 = time2 / dura2
    avgres2 = res2 / cat2

  if cat4==0: 
    avgtime4 = 0 
    avgperc4 = 0
    avgres4 = 0
  else: 
    avgtime4 = time4 / cat4
    avgperc4 = time4 / dura4
    avgres4 = res4 / cat4

  if cat1 + cat2 + cat4 == 0:
    time_total = 0
    res_total = 0
  else:
    time_total = (time1 + time2 + time4)/(cat1 + cat2 + cat4)
    res_total = (res1 + res2 + res4)/(cat1 + cat2 + cat4)

  if dura1 + dura2 + dura4 == 0:
    perc_total = 0
  else:
    perc_total = (time1 + time2 + time4)/(dura1 + dura2 + dura4)

  dict_cat = {"ExamCat1_Nr": cat1, "ExamCat2_Nr": cat2, "ExamCat4_Nr": cat4}
  dict_cat_total = {"Exams_Nr": cat1 + cat2 + cat4}
  dict_time = {"ExamCat1_AvgTime": avgtime1, "ExamCat2_AvgTime": avgtime2, "ExamCat4_AvgTime": avgtime4}
  dict_time_total = {"Exams_AvgTime": time_total}
  dict_perc = {"ExamCat1_PercTime": avgperc1, "ExamCat2_PercTime": avgperc2, "ExamCat4_PercTime": avgperc4}
  dict_perc_total = {"Exams_PercTime": perc_total}
  dict_res = {"ExamCat1_Result": avgres1, "ExamCat2_Result": avgres2, "ExamCat4_Result": avgres4}
  dict_res_total = {"Exams_Result": res_total}

  return dict_cat, dict_cat_total, dict_time, dict_time_total, dict_perc, dict_perc_total, dict_res, dict_res_total

In [ ]:
def make_exams_dataset():
  records = []
  students = src_student_info['student_id']

  for student in students:
    end_date = get_payment_date(student)
    dict_cat, dict_cat_total, dict_time, dict_time_total, dict_perc, dict_perc_total, dict_res, dict_res_total = get_exam_dictionaries(student, end_date)

    columns = {"student_id": student}
    columns.update(dict_cat)
    columns.update(dict_cat_total)
    columns.update(dict_time)
    columns.update(dict_time_total)
    columns.update(dict_perc)
    columns.update(dict_perc_total)
    columns.update(dict_res)
    columns.update(dict_res_total)
     
    records.append(columns)

  df = pd.DataFrame(records)
  return df

In [ ]:
Exams = make_exams_dataset()
Exams

,student_id,ExamCat1_Nr,ExamCat2_Nr,ExamCat4_Nr,Exams_Nr,ExamCat1_AvgTime,ExamCat2_AvgTime,ExamCat4_AvgTime,Exams_AvgTime,ExamCat1_PercTime,ExamCat2_PercTime,ExamCat4_PercTime,Exams_PercTime,ExamCat1_Result,ExamCat2_Result,ExamCat4_Result,Exams_Result
0,258798,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,258799,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,258800,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,258801,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,258802,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35225,295511,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35226,295512,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35227,295513,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35228,295514,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
Exams.to_csv(path+"exams.csv", sep="\t")

In [ ]:
for col in Exams.columns: print(Exams[col].describe())

count     35230.000000
mean     277003.640874
std       10556.258459
min      258798.000000
25%      267815.250000
50%      277001.500000
75%      286075.750000
max      295515.000000
Name: student_id, dtype: float64
count    35230.000000
mean         0.030287
std          0.559769
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         39.000000
Name: ExamCat1_Nr, dtype: float64
count    35230.00000
mean         0.22952
std          1.56513
min          0.00000
25%          0.00000
50%          0.00000
75%          0.00000
max         80.00000
Name: ExamCat2_Nr, dtype: float64
count    35230.000000
mean         0.000937
std          0.039501
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          2.000000
Name: ExamCat4_Nr, dtype: float64
count    35230.000000
mean         0.260744
std          1.924966
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max    

In [ ]:
for col in Exams.columns: print(Exams[col].value_counts())

258798    1
283091    1
283085    1
283086    1
283087    1
         ..
270900    1
270899    1
270898    1
270897    1
295515    1
Name: student_id, Length: 35230, dtype: int64
0     34972
1       110
2        51
4        15
3        12
9        11
5         7
6         7
7         6
14        6
10        5
8         5
11        5
12        5
16        4
23        2
15        2
17        2
39        1
13        1
26        1
Name: ExamCat1_Nr, dtype: int64
0     32701
1       928
2       747
3       299
4       168
5        96
6        58
7        49
8        32
9        29
10       21
13       10
15        8
14        8
16        8
20        8
11        8
12        7
21        6
30        5
17        4
18        4
25        4
26        3
22        2
33        2
34        2
24        2
29        1
80        1
70        1
76        1
35        1
71        1
31        1
19        1
45        1
53        1
46        1
Name: ExamCat2_Nr, dtype: int64
0    35208
1       11
2       11
Name:

# creating dataset: questions

In [ ]:
def get_questions_nr(student_id, end_date):
  student_questions = src_student_hub_questions[src_student_hub_questions['student_id']==student_id]
  questions_to_date = student_questions[student_questions['date_question_asked'].dt.date<end_date]
  return len(questions_to_date)

In [ ]:
def make_questions_dataset():
  records = []
  students = src_student_info['student_id']

  for student in students:
    end_date = get_payment_date(student)
    questions = get_questions_nr(student, end_date)

    columns = {"student_id": student}
    columns.update({"Questions_Nr": questions})
     
    records.append(columns)

  df = pd.DataFrame(records)
  return df

In [ ]:
Questions = make_questions_dataset()
Questions

,student_id,Questions_Nr
0,258798,0
1,258799,0
2,258800,0
3,258801,0
4,258802,0
...,...,...
35225,295511,0
35226,295512,0
35227,295513,0
35228,295514,0


In [ ]:
for col in Questions.columns: print(Questions[col].describe())

count     35230.000000
mean     277003.640874
std       10556.258459
min      258798.000000
25%      267815.250000
50%      277001.500000
75%      286075.750000
max      295515.000000
Name: student_id, dtype: float64
count    35230.000000
mean         0.003633
std          0.091281
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          9.000000
Name: Questions_Nr, dtype: float64


In [ ]:
for col in Questions.columns: print(Questions[col].value_counts())

258798    1
283091    1
283085    1
283086    1
283087    1
         ..
270900    1
270899    1
270898    1
270897    1
295515    1
Name: student_id, Length: 35230, dtype: int64
0    35139
1       73
2       10
3        3
4        3
9        1
5        1
Name: Questions_Nr, dtype: int64


In [ ]:
Questions.to_csv(path+"questions.csv", sep="\t")

# creating dataset: timing

In [ ]:
def get_periods(student_id, end_date):
  date_registered = src_student_info[src_student_info['student_id']==student_id]['date_registered'].dt.date.values[0]
  diff = end_date - date_registered
  period_registered = max(diff.days, 0)

  date_first_course = src_student_learning[src_student_learning['student_id']==student_id]['date_watched'].min().date()
  if pd.isnull(date_first_course):
    period_first_course = 0
  else:
    diff = end_date - date_first_course
    period_first_course = max(diff.days, 0)
  
  date_last_course = src_student_learning[src_student_learning['student_id']==student_id]['date_watched'].max().date()
  if pd.isnull(date_last_course):
    period_last_course = 0
  else:
    diff = end_date - date_last_course
    period_last_course = max(diff.days, 0)

  student_engagements = src_student_engagement[src_student_engagement["student_id"]==student_id]
  student_quizzes = student_engagements[student_engagements["engagement_quizzes"]>0]
  date_first_quiz = student_quizzes['date_engaged'].min().date()
  if pd.isnull(date_first_quiz):
    period_first_quiz = 0
  else:
    diff = end_date - date_first_quiz
    period_first_quiz = max(diff.days, 0)

  date_last_quiz = student_quizzes['date_engaged'].max().date()
  if pd.isnull(date_last_quiz):
    period_last_quiz = 0
  else:
    diff = end_date - date_last_quiz
    period_last_quiz = max(diff.days, 0)

  date_first_exam = src_student_exams[src_student_exams['student_id']==student_id]['date_exam_completed'].min().date()
  if pd.isnull(date_first_exam):
    period_first_exam = 0
  else:
    diff = end_date - date_first_exam
    period_first_exam = max(diff.days, 0)

  date_last_exam = src_student_exams[src_student_exams['student_id']==student_id]['date_exam_completed'].max().date()
  if pd.isnull(date_last_exam):
    period_last_exam = 0
  else:
    diff = end_date - date_last_exam
    period_last_exam = max(diff.days, 0)

  date_first_question = src_student_hub_questions[src_student_hub_questions['student_id']==student_id]['date_question_asked'].min().date()
  if pd.isnull(date_first_question):
    period_first_question = 0
  else:
    diff = end_date - date_first_question
    period_first_question = max(diff.days, 0)

  date_last_question = src_student_hub_questions[src_student_hub_questions['student_id']==student_id]['date_question_asked'].max().date()
  if pd.isnull(date_last_question):
    period_last_question = 0
  else:
    diff = end_date - date_last_question
    period_last_question = max(diff.days, 0)

  periods_campaigns = []

  date_winter_sale = datetime.date(2022, 1, 17)
  diff = end_date - date_winter_sale
  period_winter_sale = max(diff.days, 0)
  if period_winter_sale>0: periods_campaigns.append(period_winter_sale)

  date_spring_offer = datetime.date(2022, 3, 21)
  diff = end_date - date_spring_offer
  period_spring_offer = max(diff.days, 0)
  if period_spring_offer>0: periods_campaigns.append(period_spring_offer)

  date_summer_campaign = datetime.date(2022, 5, 20)
  diff = end_date - date_summer_campaign
  period_summer_campaign = max(diff.days, 0)
  if period_summer_campaign>0: periods_campaigns.append(period_summer_campaign)

  date_wanted_campaign = datetime.date(2022, 7, 18)
  diff = end_date - date_wanted_campaign
  period_wanted_campaign = max(diff.days, 0)
  if period_wanted_campaign>0: periods_campaigns.append(period_wanted_campaign)

  date_gamification = datetime.date(2022, 9, 16)
  diff = end_date - date_gamification
  period_gamification = max(diff.days, 0)
  if period_gamification>0: periods_campaigns.append(period_gamification)

  date_hero_campaign = datetime.date(2022, 9, 19)
  diff = end_date - date_hero_campaign
  period_hero_campaign = max(diff.days, 0)
  if period_hero_campaign>0: periods_campaigns.append(period_hero_campaign)

  if len(periods_campaigns)==0:
    period_campaigns = 0
  else: 
    period_campaigns = min(periods_campaigns) # from the closest campaign

  return period_registered, period_first_course, period_last_course, period_first_quiz, period_last_quiz, period_first_exam, period_last_exam, period_first_question, period_last_question, period_campaigns


In [ ]:
def make_timing_dataset():
  records = []
  students = src_student_info['student_id']

  for student in students:
    end_date = get_payment_date(student)
    periods = get_periods(student, end_date)

    columns = {"student_id": student}
    columns.update({"Period_Reg": periods[0]})
    columns.update({"Period_FCourse": periods[1]})
    columns.update({"Period_LCourse": periods[2]})
    columns.update({"Period_FQuiz": periods[3]})
    columns.update({"Period_LQuiz": periods[4]})
    columns.update({"Period_FExam": periods[5]})
    columns.update({"Period_LExam": periods[6]})
    columns.update({"Period_FQuest": periods[7]})
    columns.update({"Period_LQuest": periods[8]})
    columns.update({"Period_Camp": periods[9]})

    records.append(columns)

  df = pd.DataFrame(records)
  return df

In [ ]:
Timing = make_timing_dataset()
Timing

,student_id,Period_Reg,Period_FCourse,Period_LCourse,Period_FQuiz,Period_LQuiz,Period_FExam,Period_LExam,Period_FQuest,Period_LQuest,Period_Camp
0,258798,364,364,364,0,0,0,0,0,0,103
1,258799,364,0,0,0,0,0,0,0,0,103
2,258800,0,0,0,0,0,0,0,0,0,0
3,258801,364,0,0,0,0,0,0,0,0,103
4,258802,364,0,0,0,0,0,0,0,0,103
...,...,...,...,...,...,...,...,...,...,...,...
35225,295511,72,72,72,72,72,0,0,0,0,103
35226,295512,72,72,72,72,72,0,0,0,0,103
35227,295513,72,72,72,0,0,0,0,0,0,103
35228,295514,72,0,0,0,0,0,0,0,0,103


In [ ]:
for col in Timing.columns: print(Timing[col].describe())

count     35230.000000
mean     277003.640874
std       10556.258459
min      258798.000000
25%      267815.250000
50%      277001.500000
75%      286075.750000
max      295515.000000
Name: student_id, dtype: float64
count    35230.000000
mean       198.211865
std         94.992953
min          0.000000
25%        129.000000
50%        195.000000
75%        281.000000
max        364.000000
Name: Period_Reg, dtype: float64
count    35230.000000
mean        93.551433
std        117.000248
min          0.000000
25%          0.000000
50%          0.000000
75%        180.000000
max        364.000000
Name: Period_FCourse, dtype: float64
count    35230.000000
mean        89.880869
std        114.047948
min          0.000000
25%          0.000000
50%          0.000000
75%        169.000000
max        364.000000
Name: Period_LCourse, dtype: float64
count    35230.000000
mean        34.160914
std         84.018930
min          0.000000
25%          0.000000
50%          0.000000
75%          0.0

In [ ]:
for col in Timing.columns: print(Timing[col].value_counts())

258798    1
283091    1
283085    1
283086    1
283087    1
         ..
270900    1
270899    1
270898    1
270897    1
295515    1
Name: student_id, Length: 35230, dtype: int64
0      1019
138     709
137     474
212     425
136     262
       ... 
26        1
64        1
65        1
70        1
59        1
Name: Period_Reg, Length: 363, dtype: int64
0      18298
138      412
137      289
136      196
1        188
       ...  
33         1
43         1
64         1
34         1
48         1
Name: Period_FCourse, Length: 352, dtype: int64
0      19074
136      291
138      280
135      256
137      252
       ...  
56         1
2          1
9          1
8          1
5          1
Name: Period_LCourse, Length: 303, dtype: int64
0      28924
138      157
1        145
137      128
136      104
       ...  
69         1
50         1
58         1
70         1
26         1
Name: Period_FQuiz, Length: 347, dtype: int64
0      29413
136      156
137      116
138      100
135       94
       ...

In [ ]:
Timing.to_csv(path+"timing.csv", sep="\t")

# creating final dataset: students

In [ ]:
def make_students_dataset(): 
  records = []
  students = src_student_info['student_id']

  print("making students dataframe")
  for student in students:
    end_date = get_payment_date(student)
    if end_date < datetime.date(2022,12,31):
      paying = 1
    else:
      paying = 0

    columns = {"student_id": student, "Paying": paying}
    records.append(columns)

  df_students = pd.DataFrame(records)

  files = ['courses.csv', 'quizes.csv', 'engagements.csv', 'exams.csv', 'questions.csv', 'timing.csv']
  for file in files:
    print("merging with "+file)
    df = pd.read_csv(path+file, sep="\t")
    df_students = df_students.merge(df.iloc[:,1:], left_on='student_id', right_on='student_id')

  return df_students

In [ ]:
df_students = make_students_dataset()

making students dataframe
merging with courses.csv
merging with quizes.csv
merging with engagements.csv
merging with exams.csv
merging with questions.csv
merging with timing.csv


In [ ]:
df_students.shape

(35230, 221)

In [ ]:
for col in df_students.columns: print(df_students[col].describe())

count     35230.000000
mean     277003.640874
std       10556.258459
min      258798.000000
25%      267815.250000
50%      277001.500000
75%      286075.750000
max      295515.000000
Name: student_id, dtype: float64
count    35230.000000
mean         0.060602
std          0.238602
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: Paying, dtype: float64
count    35230.000000
mean         0.011013
std          0.104366
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: Course_Nr_2, dtype: float64
count    35230.000000
mean         0.009991
std          0.099458
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: Course_Nr_3, dtype: float64
count    35230.000000
mean         0.004485
std          0.066819
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max 

In [ ]:
for col in df_students.columns: print(df_students[col].value_counts())

258798    1
283091    1
283085    1
283086    1
283087    1
         ..
270900    1
270899    1
270898    1
270897    1
295515    1
Name: student_id, Length: 35230, dtype: int64
0    33095
1     2135
Name: Paying, dtype: int64
0    34842
1      388
Name: Course_Nr_2, dtype: int64
0    34878
1      352
Name: Course_Nr_3, dtype: int64
0    35072
1      158
Name: Course_Nr_4, dtype: int64
0    35108
1      122
Name: Course_Nr_5, dtype: int64
0    27802
1     7428
Name: Course_Nr_7, dtype: int64
0    35021
1      209
Name: Course_Nr_11, dtype: int64
0    35090
1      140
Name: Course_Nr_12, dtype: int64
0    34707
1      523
Name: Course_Nr_13, dtype: int64
0    34142
1     1088
Name: Course_Nr_14, dtype: int64
0    34868
1      362
Name: Course_Nr_15, dtype: int64
0    34554
1      676
Name: Course_Nr_16, dtype: int64
0    35128
1      102
Name: Course_Nr_19, dtype: int64
0    34857
1      373
Name: Course_Nr_20, dtype: int64
0    34055
1     1175
Name: Course_Nr_21, dtype: int64
0    349

In [ ]:
df_students

,student_id,Paying,Course_Nr_2,Course_Nr_3,Course_Nr_4,Course_Nr_5,Course_Nr_7,Course_Nr_11,Course_Nr_12,Course_Nr_13,...,Period_Reg,Period_FCourse,Period_LCourse,Period_FQuiz,Period_LQuiz,Period_FExam,Period_LExam,Period_FQuest,Period_LQuest,Period_Camp
0,258798,0,0,0,0,0,0,0,0,0,...,364,364,364,0,0,0,0,0,0,103
1,258799,0,0,0,0,0,0,0,0,0,...,364,0,0,0,0,0,0,0,0,103
2,258800,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,258801,0,0,0,0,0,0,0,0,0,...,364,0,0,0,0,0,0,0,0,103
4,258802,0,0,0,0,0,0,0,0,0,...,364,0,0,0,0,0,0,0,0,103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35225,295511,0,0,0,0,0,1,0,0,0,...,72,72,72,72,72,0,0,0,0,103
35226,295512,0,0,0,0,0,1,0,0,0,...,72,72,72,72,72,0,0,0,0,103
35227,295513,0,0,0,0,0,0,0,0,0,...,72,72,72,0,0,0,0,0,0,103
35228,295514,0,0,0,0,0,0,0,0,0,...,72,0,0,0,0,0,0,0,0,103


In [ ]:
df_students.to_csv(path+"Students.csv", sep="\t")